# Code for Lecture-3 of Short Course of Temporal Point Processes

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# fix random seed 
np.random.seed(12345)
torch.random.manual_seed(12345)

## Neural Hawkes Process

### Continuous-time LSTM cell

The LSTM cell $c(t)$ drifts from $c_{\text{start}}$ towards $c_{\text{target}}$. 

In [3]:
class CTLSTMCell(nn.Module): 
    
    def __init__(self, hdim): 
        super(CTLSTMCell, self).__init__()
        """
        hdim : # of hidden neurons
        """
        self.hdim = hdim 
        self.lin = nn.Linear(hdim*2, hdim*6, bias=True)
        return 
    
    def forward(self, x, h, c, ct): 
        """
        x : input embedding
        h : hidden state right before time t 
        c : LSTM cell right before time t 
        ct : LSTM target cell given current history
        """
        x = torch.cat((x, h), dim=0)
        y = self.lin(x)
        
        gi, gf, z, git, gft, gd = y.chunk(6, 0)
        
        gi = torch.sigmoid(gi)
        gf = torch.sigmoid(gf)
        z = torch.tanh(z)
        git = torch.sigmoid(git)
        gft = torch.sigmoid(gft)
        gd = F.softplus(gd)
        
        cs = gf * c + gi * z 
        ct = gft * ct + git * z
        
        return cs, ct, gd
    
    def decay(self, cs, ct, gd, dt): 
        """
        cs : LSTM start cell
        ct : LSTM target cell 
        gd : decay gate
        dt : elapsed time 
        """
        c = ct + (cs - ct) * torch.exp(-gd * dt)
        h = torch.tanh(c)
        
        return c, h

### Neural Hawkes process

The intensity is defined as $\lambda_k(t) = \text{Softplus}(\text{Linear}(h(t)))$.  

In [4]:
class NHP(nn.Module): 
    
    def __init__(self, kdim, hdim): 
        super(NHP, self).__init__()
        """
        kdim : # of event types 
        hdim : # of hidden neurons
        """
        self.eps = np.finfo(float).eps 
        self.max = np.finfo(float).max 
        self.kdim = kdim 
        self.hdim = hdim 
        self.BOS = kdim 
        
        self.emb_in = nn.Embedding(kdim+1, hdim)
        self.ctlstm = CTLSTMCell(hdim)
        self.emb_out = nn.Linear(hdim, kdim)
        
        self.cs = torch.zeros(size=[hdim], dtype=torch.float32)
        self.ct = torch.zeros(size=[hdim], dtype=torch.float32)
        self.gd = torch.zeros(size=[hdim], dtype=torch.float32)
        
        return 
    
    def start(self): 
        self.cs = torch.zeros(size=[hdim], dtype=torch.float32)
        self.ct = torch.zeros(size=[hdim], dtype=torch.float32)
        self.gd = torch.zeros(size=[hdim], dtype=torch.float32)
        self.update(self.BOS, 0.0)
        return 
    
    def update(self, k, dt): 
        """
        k : event type 
        dt : elapsed time since last event
        """
        c, h = self.ctlstm.decay(self.cs, self.ct, self.gd, dt)
        x = self.emb_in(torch.LongTensor([k]))[0]
        self.cs, self.ct, self.gd = self.ctlstm(x, h, c, self.ct)
        
        return 
    
    def forward(self, k, dt): 
        self.update(k, dt)
        return 
    
    def compute_intensities(self, dt): 
        c, h = self.ctlstm.decay(self.cs, self.ct, self.gd, dt)
        return F.softplus(self.emb_out(h))
    
    def compute_total_intensity(self, dt): 
        intensities = self.compute_intensities(dt)
        return torch.sum(intensities)

### Draw a sequence of events by thinning algorithm

For the code to be easy to understand, I only have non-vectorized implementation. Please check the repos for my published papers for highly vectorized and optimized implementation. 

In [5]:
def thinning(model): 
    dt = 0.0
    bound = 100.0 
    # manualy chosen for simplicity
    # in principle, it can be found using the method in Appendix B.3 of Mei & Eisner 2017
    while True: 
        u = np.random.uniform(0.0, 1.0)
        dt += -np.log(1-u) / bound
        intens = model.compute_intensities(dt)
        total_inten = torch.sum(intens)
        accept_prob = total_inten / bound
        u = np.random.uniform(0.0, 1.0)
        if u <= accept_prob: 
            break 
    
    k = torch.multinomial(intens, 1)
    
    return k, dt
    

Draw data from a low-entropy distribution: (1) draw $dt$ from a univariate NHP; (2) draw $k$ from a n-gram model.

In [11]:
kdim = 32 
hdim = 8
nhp = NHP(1, hdim)
# init by BOS 
nhp.start()

T = 100.0
t = 0
seq = []
CONTEXT = 0

while True:
    # draw dt using thinning algorithm
    
    _, dt = thinning(nhp)
    t += dt
    if t <= T: 
        k = (CONTEXT + 1) % kdim
        seq += [(dt, k)] # track dt, not t, easy to use
        # update model 
        nhp.forward(0, dt)
        # update CONTEXT 
        CONTEXT = k
    else: 
        break

print(f"over time interval [0, {T}]")
print(f"# of events : {len(seq)}")

over time interval [0, 100.0]
# of events : 78


### Train NHP by MLE

In [21]:
def mle(data, model): 
    """
    compute log-likelihood of seq under model
    """
    J = 10
    model.start()
    loglik = 0.0
    
    for event in seq: 
        dt, k = event
        # log intensity 
        loglik += torch.log(model.compute_intensities(dt)[k])
        # integral
        integral = 0.0 
        for j in range(J): 
            # draw uniform-distributed time points
            dtj = np.random.uniform(0.0, dt)
            integral += model.compute_total_intensity(dtj)
        integral /= J 
        integral *= dt 
        loglik -= integral
        # update model
        model.forward(k, dt)
    
    return loglik

In [22]:
import time
import torch.optim as optim

model = NHP(kdim, hdim)
sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

MAX_ITER = 10

for i in range(MAX_ITER): 
    
    tic = time.time()
    sgd.zero_grad()
    loglik = mle(seq, model)
    loss = -loglik 
    loss.backward()
    sgd.step()
    toc = time.time()
    
    print(f"Iter-{i}: log-likelihood = {float(loglik):.4f}; time cost = {toc-tic:.4f}")

Iter-0: log-likelihood = -2213.8076; time cost = 0.2930
Iter-1: log-likelihood = -2118.6753; time cost = 0.2883
Iter-2: log-likelihood = -1947.0254; time cost = 0.2890
Iter-3: log-likelihood = -1701.9241; time cost = 0.2909
Iter-4: log-likelihood = -1387.6753; time cost = 0.2914
Iter-5: log-likelihood = -1055.1245; time cost = 0.2903
Iter-6: log-likelihood = -770.9929; time cost = 0.2888
Iter-7: log-likelihood = -568.3903; time cost = 0.2893
Iter-8: log-likelihood = -449.9092; time cost = 0.2909
Iter-9: log-likelihood = -391.9956; time cost = 0.2875


### Train NHP by NCE

In [72]:
class SimpleNoise(nn.Module): 
    """
    a simple noise distribution
    """
    
    def __init__(self, kdim, total_intensity): 
        super(SimpleNoise, self).__init__()
        """
        kdim : # of event types 
        hdim : # of hidden neurons
        """
        self.total_intensity = total_intensity
        self.inten = total_intensity / kdim
        self.intens = torch.zeros([kdim], dtype = torch.float32).fill_(self.inten)
        return 
    
    def start(self): 
        # do nothing
        return 
    
    def update(self, k, dt): 
        """
        k : event type 
        dt : elapsed time since last event
        """
        # does nothing
        # simple distribution, no dependence on history
        return 
    
    def forward(self, k, dt): 
        self.update(k, dt)
        return 
    
    def compute_intensities(self, dt): 
        return self.intens
    
    def compute_total_intensity(self, dt): 
        return self.total_intensity
    
    def draw(self, dt): 
        noise_events = []
        dtj = 0.0
        while True: 
            # draw noise time
            u = np.random.uniform(0.0, 1.0)
            dtj += -np.log(1-u) / self.total_intensity
            if dtj <= dt: 
                # draw noise type
                kj = torch.multinomial(self.intens, 1)[0]
                noise_events += [(dtj, kj)]
            else: 
                break 
        return noise_events

In [70]:
def nce(data, model, noise): 
    """
    compute log-likelihood of seq under model
    """
    model.start()
    noise.start()
    loglik = 0.0
    
    for event in seq: 
        dt, k = event
        
        # real event & noise non-event
        p_real = model.compute_intensities(dt)[k]
        q_real = noise.compute_intensities(dt)[k]
        loglik += torch.log(p_real / (p_real + q_real))
        
        # real non-event & noise event
        for dtj, kj in noise.draw(dt): 
            #print(f"dtj = {dtj}")
            #print(f"kj = {kj}")
            p_noise = model.compute_intensities(dtj)[kj]
            q_noise = noise.compute_intensities(dtj)[kj]
            loglik -= torch.log(p_noise + q_noise)
        
        # update model and noise with real history
        model.forward(k, dt)
        noise.forward(k, dt)
    
    return loglik

In [ ]:
model = NHP(kdim, hdim)
noise = SimpleNoise(kdim, len(seq)*1.0/T)

sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

MAX_ITER = 50

for i in range(MAX_ITER): 
    
    tic = time.time()
    sgd.zero_grad()
    loglik = nce(seq, model, noise)
    loss = -loglik 
    loss.backward()
    sgd.step()
    toc = time.time()
    
    loglik = mle(seq, model)
    
    print(f"Iter-{i}: log-likelihood = {float(loglik):.4f}; time cost = {toc-tic:.4f}")

Iter-0: log-likelihood = -2229.4138; time cost = 0.1280
Iter-1: log-likelihood = -2222.2319; time cost = 0.1171
Iter-2: log-likelihood = -2211.9751; time cost = 0.1228
Iter-3: log-likelihood = -2199.3914; time cost = 0.1178
Iter-4: log-likelihood = -2185.8748; time cost = 0.1093
Iter-5: log-likelihood = -2170.5784; time cost = 0.1091
Iter-6: log-likelihood = -2153.3032; time cost = 0.1190
Iter-7: log-likelihood = -2134.7344; time cost = 0.1110
Iter-8: log-likelihood = -2115.0427; time cost = 0.1154
Iter-9: log-likelihood = -2094.1021; time cost = 0.1165
Iter-10: log-likelihood = -2071.9785; time cost = 0.1173
Iter-11: log-likelihood = -2049.8616; time cost = 0.1133
Iter-12: log-likelihood = -2026.9714; time cost = 0.1117
Iter-13: log-likelihood = -2003.6262; time cost = 0.1200
Iter-14: log-likelihood = -1979.0774; time cost = 0.1128
Iter-15: log-likelihood = -1954.2950; time cost = 0.1180
Iter-16: log-likelihood = -1929.0826; time cost = 0.1103
Iter-17: log-likelihood = -1902.9490; tim

### Predict next event time and type by sampling (approx. MBR)

In [68]:
def predict_time(model): 
    dts, ks = [], []
    n = 10 
    k, dt = thinning(model)
    dts += [float(dt)]
    dt_pred = np.mean(dts)
    return dt_pred

def predict_type(model, dt): 
    intens = model.compute_intensities(dt)
    k_pred = torch.argmax(intens)
    return k_pred

In [69]:
se = 0.0
nerr = 0

nhp.start() # restart
n = 100

for i, s in enumerate(seq[:n]): 
    # predict
    dt_pred = predict_time(nhp)
    # time
    dt = seq[i][0]
    se += (dt_pred - dt) ** 2
    # type 
    k_pred = predict_type(nhp, dt)
    k = seq[i][1]
    if k_pred != k: 
        nerr += 1

print(f"check time prediction accuracy")
#print(f"RMSE using estimated intensity : {rmse_mle:.4f}")
print(f"RMSE using true model : {np.sqrt(se/n):.4f}")

print(f"\ncheck type prediction error rate")
#print(f"Error Rate using estimated intensities : {100.0*nerr_mle/len(seq):.2f}%")
print(f"Error Rate using true model : {100.0*nerr/n:.2f}%")


check time prediction accuracy
RMSE using true model : 0.4117

check type prediction error rate
Error Rate using true model : 73.00%
